In [1]:
from selenium import webdriver
import re
import time
import requests
from datetime import datetime  
from datetime import timedelta 

options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--incognito')
options.add_argument('--headless')
# your location of chromedriver.exe
# the version of chromedriver should be same as of browser
driver = webdriver.Chrome(r"C:\Users\Lenovo\Downloads\A\chromedriver.exe", options=options)

url1 = 'https://www.dream11.com/leagues'
driver.get(url1)
# Take a second to load javascript
time.sleep(2)
content = driver.page_source

#### Name of tournament 

In [2]:
tournament_pattern = r'(matchCardHeaderTitle_c5373 matchCardHeaderTitleDesktop_a2024">)(\w*-*\'*\s*\w*-*\'*\s*\w*-*\'*\s*\w*-*\'*\s*\w*-*\'*\s*\w*-*\w*\'*\s*\w*)'
tournament_data = re.findall(tournament_pattern,content)

#### Match Name

In [3]:
lteam_pattern = r'(<div class="squadShortName_a116b squadShortNameLeft_db179">)([A-Z]*-*[A-Z])'
lteam_data = re.findall(lteam_pattern,content)

In [4]:
rteam_pattern = r'(<div class="squadShortName_a116b squadShortNameRight_42ab0">)([A-Z]*-*[A-Z*])'
rteam_data = re.findall(rteam_pattern,content)

In [5]:
match_name = list()
for i in range(len(tournament_data)):
    match_name.append(lteam_data[i][1] + ' vs ' + rteam_data[i][1])

#### Match Time Remaining

In [6]:
time_pattern = r'(matchCardTimerDesktop_48a55"><div>)(\w*\s*\w*\s*\w*)'
time_data = re.findall(time_pattern,content)

#### Match Start Time

In [7]:
days = 0
hours = 0
minutes = 0
dates = list()
for i in range(len(tournament_data)):
    if (time_data[i][1].rfind('d')) != -1:
        days = time_data[i][1][0]
        start_date = datetime.now() + timedelta(days=int(days))
        dates.append(start_date)

    elif (time_data[i][1].rfind('h')) != -1:
        hours = int(time_data[i][1].split('h')[0])
        start_date = datetime.now() + timedelta(hours=int(hours))

        if (time_data[i][1].rfind('m')) != -1:
            minutes = int(time_data[i][1].split('m')[0].split(' ')[1])
            start_date = start_date + timedelta(minutes=int(minutes))
        dates.append(start_date)
    
    elif (time_data[i][1].rfind('m')) != -1:
        minutes = int(time_data[i][1].split('m')[0])
        start_date = datetime.now() + timedelta(minutes=int(minutes))
        dates.append(start_date)

#### Match Status

In [8]:
match_status = list()
for i in range(len(tournament_data)):
    if datetime.now() > dates[i]:
        match_status.append("Already Started")
    else:
        match_status.append("Not Started")

#### Building Final Data

In [9]:
data = list()
data_dict = dict()
for i in range(len(tournament_data)):
    data.append({'tournament_name':tournament_data[i][1], 'match_name':match_name[i], 'match_status':match_status[i], 'start_time':str(dates[i])})

data_dict = {'tournament_data':data}

#### Parsing to json

In [10]:
import json
import csv
dream11_json_data = json.dumps(data_dict)

In [11]:
data = json.loads(dream11_json_data)
dream11_data = data['tournament_data']

#### Saving it as CSV file

In [31]:
data_file = open('data_file1.csv', 'w')

csv_writer = csv.writer(data_file)
# writing header to a file
header = data.keys() 
csv_writer.writerow(header)
for data in dream11_data:   
    # Writing data of CSV file 
    csv_writer.writerow(data.values()) 
  
data_file.close() 

#### Displaying csv file

In [32]:
with open('data_file.csv') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=' ')
    for row in spamreader:
        print(' '.join(row))

tournament_name,match_name,match_status,start_time

FanCode ECS T10 - Barcelona,HIS vs PIC,Not Started,2021-03-09 20:59:52.956255

Australia One Day Cup,VCT vs TAS,Not Started,2021-03-10 04:25:52.956255

Kodak Kerala T20,LIO vs EAG,Not Started,2021-03-10 09:25:52.956255

Afghanistan vs Zimbabwe Test,AFG vs ZIM,Not Started,2021-03-10 10:25:52.956255

FanCode ECS T10 - Barcelona,BAG vs KHA,Not Started,2021-03-10 12:25:52.956255

Kodak Kerala T20,ROY vs TUS,Not Started,2021-03-10 13:25:52.956255

FanCode ECS T10 - Barcelona,SKY vs KHA,Not Started,2021-03-10 14:25:52.956255

FanCode ECS T10 - Barcelona,BAK vs BAG,Not Started,2021-03-10 16:25:52.956255

West Indies vs Sri Lanka ODI,WI vs SL,Not Started,2021-03-10 18:25:52.956255

Road Safety T20 World Series,BD-L vs SL-L,Not Started,2021-03-10 18:25:52.956255

FanCode ECS T10 - Barcelona,BAG vs RAS,Not Started,2021-03-10 18:25:52.956255

FanCode ECS T10 - Barcelona,BAK vs MIN,Not Started,2021-03-10 20:25:52.956255

Plunket Shield,CTB vs OV,